## Battle of the Neighborhoods - Coursera Capstone Project

### Introduction

**In this Project, there will be an analysis on neighborhoods from Toronto, Canada and New York City. Due to globalization, people are starting businesses all over the world. For instance, there are people opening up donut shops in China. Meanwhile, there are Chinese people try to open Chinese restaurants overseas. A lot of people are targeting at Toronto and New York City. People will like to see which of the city is more competitive so that they could go to the other one to have their businesses settled down. This project will make comparisons between the two cities, and it will give Chinese people some ideas about where it the better place to open up a Chiense Restaurant.**

### Problem

**The purpose of this project is trying to find out which city is better to open up a Chinese restaurant between Toronto and New York.**

1. Visualize neighbourhoods in Toronto and New York City and try to find the best neighbourhoods that are the most popular in the cities.
2. Compare the picked two neighbourhoods.
3. Make the final decision on which one of the two cities is good for opening up a Chinese restaurant.

### Libraries

1. Pandas: For creating and manipulating dataframes.

2. Folium: Python visualization library would be used to visualize the neighborhoods cluster distribution of using interactive leaflet map.

3. Scikit Learn: For importing k-means clustering.

4. JSON: Library to handle JSON files.

5. XML: To separate data from presentation and XML stores data in plain text format.

6. Geocoder: To retrieve Location Data.

7. Beautiful Soup and Requests: To scrap and library to handle http requests.

8. Matplotlib: Python Plotting Module.

### Data

For this project we need the following data:

1. GeoSpace data.
2. New York City data that contains list Boroughs, Neighbourhoods along with their latitude and longitude.
3. Toronto data that contains list Boroughs, Neighbourhoods along with their latitude and longtitude.